In [1]:
import psycopg2 as ps
import datetime as dt

In [7]:
import psycopg2
import pandas as pd
import numpy as np

class DatabaseOperations:
    def __init__(self):
        self.connection = psycopg2.connect(database="insititue", 
                                           user="postgres", 
                                           password="Mukesh@4912")
        self.cursor = self.connection.cursor()
        print("Connection established successfully.")

    def create_table(self):
        # Drop a table if it exists.
        query_drop = "DROP TABLE IF EXISTS student"
        self.cursor.execute(query_drop)
        print("The table is dropped.")

        # Create a table
        query_create = "CREATE TABLE student (prn INT, name VARCHAR(39), email VARCHAR(50))"
        self.cursor.execute(query_create)
        print("The table is created.")
    
    # Insert multiple records
    def multiple_inserts(self):
        values = [(1, 'Heena', 'heena@sig.ac.in'),
                  (2, 'Geeta', 'geeta@sig.ac.in'),
                  (3, 'Sita', 'sita@sig.ac.in')]
        args = ','.join(self.cursor.mogrify("(%s,%s,%s)", i).decode('utf-8') for i in values)
        self.cursor.execute("INSERT INTO student VALUES" + args)
        print("Records inserted")
        
    # Search for a record 
    def search_record(self, search_criteria):
        query = "SELECT * FROM student WHERE prn = %s"
        self.cursor.execute(query, search_criteria)
        records = self.cursor.fetchall()
        if records:
            print("Search results:")
            for record in records:
                print(record)

    # Delete records
    def delete_record(self, fname):
        query = "DELETE FROM student WHERE name = %s"
        self.cursor.execute(query, (fname,))
        self.connection.commit()
        print("Records deleted.")

    # Execute many query
    def execute_many(self, table, df):
        # Using cursor.executemany() to insert the dataframe

        # Create a list of tuples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        print("The data from csv file is")
        print(tuples)

        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        print("Columns are", cols)

        # SQL query to execute
        query = "INSERT INTO %s(%s) VALUES (%s, %s, %s)" % (table, cols, '%s', '%s', '%s')

        try:
            self.cursor.executemany(query, tuples)
            self.connection.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            self.connection.rollback()
            self.cursor.close()
            return 1

        print("executemany() done")

# cre
db_operations = DatabaseOperations()
db_operations.create_table()
db_operations.multiple_inserts()
db_operations.search_record((3,))
db_operations.delete_record('Sita')

# dataframe for execution (replace it with your actual dataframe)
data = {'prn': [4, 5], 'name': ['Rachana', 'Ritesh'], 'email': ['rachana@email.com', 'ritesh@email.com']}
df = pd.DataFrame(data)

# Uncomment and provide the appropriate DataFrame for execution
db_operations.execute_many('student', df)

# Close connection outside the class methods
db_operations.connection.commit()
db_operations.connection.close()

Connection established successfully.
The table is dropped.
The table is created.
Records inserted
Search results:
(3, 'Sita', 'sita@sig.ac.in')
Records deleted.
The data from csv file is
[(4, 'Rachana', 'rachana@email.com'), (5, 'Ritesh', 'ritesh@email.com')]
Columns are prn,name,email
executemany() done


In [9]:
#Converting the database into csv file
import pandas as pd
csv_file = "data1.csv"
df = pd.read_csv(csv_file)
df = df.rename(columns = {
    "PRN" : "prn" ,
    "Name" : "name" ,
    "Email": "email" 
})
print(df)

   prn     name              email
0    1    Heena    heena@sig.ac.in
1    2    Geeta    geeta@sig.ac.in
2    4  Rachana  rachana@email.com
3    5   Ritesh   ritesh@email.com
